# 2.1.1 Functions

This is a small introduction to functions from the point of view of functional programming _and_ software engineering. It ends with a discussion about the basic way in which functions are represented in an object-oriented programming language like Scala. 

### References

_[Optional]_ __[Why Functional Programming Matters](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf)__ John Hughes. This is a classic paper that motivates the need for functional programming by appealing to software enginerring principles such as modularity. It's written using the Miranda programming language, and it's more an academic paper that undergraduate material. Nevertheless, its reading is highly recommeded. Give it a try if you find some time!

__[Scala book (online)](https://docs.scala-lang.org/overviews/scala-book/introduction.html)__.

- [Pure functions](https://docs.scala-lang.org/overviews/scala-book/pure-functions.html)

## What are (pure) functions?

Functions are computational devices that transform input _values_ into output _values_, and do nothing _else_.

In [1]:
// `add one` function

def addOne(x: Int): Int = 
    x+1

defined function addOne

If we run this function, the only thing that happens is the computation of a new value:

In [2]:
addOne(5)

res1: Int = 6

In [3]:
var i: Int = 1

i: Int = 3

In [4]:
i

res3: Int = 1

In [5]:
i = 3

In [6]:
i

res5: Int = 3

In [7]:
val a: Int = 1

a: Int = 1

In [7]:
a = 3

cmd7.sc:1: reassignment to val
val res7 = a = 3
             ^Compilation Failed

: 

Functions that do something else, besides returning values, are called _impure_ functions. Functional programming deals only with _pure_, or mathematical, functions.

In [10]:
// An impure function

def impureAdd(input: Int): Int = {
    println("sumando 1 a " + input)
    input + 1
}


defined function impureAdd

If we run this function, we will see an _effect_ in the console (besides the pure computation of `input + 1`): 

In [11]:
impureAdd(5)

sumando 1 a 5


res10: Int = 6

There are many kinds of effects: writing to the console, reading from the keyworkd, reading from a socket, calling a web service, executing a query over the database, etc. Clearly, we need effects if we want our programs to do something useful, so pure functions alone are not enough. We will talk about this later on.



## Functions as modularity devices

Why are functions so important in programming? Because they help us to _modularize_ our code. For instance, let's consider the following programs, which access the following data structure of key-value pairs (we will talk about this structure in detail later on):

In [13]:
val config: Map[String, String] = 
    Map("URL" -> "http://hablapps.com",
        "PORT" -> "8080")

config: Map[String, String] = Map(
  "URL" -> "http://hablapps.com",
  "PORT" -> "8080"
)

Our first program access the configuration data for the value of the "URL" key. If it's not found, then the default value "default.url" is returned (similarly, we will discuss the `match` keyword further in the course).

In [2]:
// Program 1
val url: String = config.get("URL") match {
  case Some(u) => u
  case None => "default.url"
}

url: String = "http://hablapps.com"

Our second program accesses the configuration data for the value of the "PORT" key. If it's not found, then the default value "8080" is returned.

In [3]:
// Program 2
val port: String = config.get("PORT") match {
  case Some(p) => p
  case None => "8080"
}

port: String = "8080"

In [3]:
// Program 2
val port: String = config.get("DBNAME") match {
  case Some(p) => p
  case None => "test"
}

port: String = "8080"

These two programs do _almost_ the same. The only differences lie in the particular keys and default values the programs refer to, but, otherwise, they do the same thing. However, this _common factor_ is not reflected in the code. Indeed, we may get one program from the other by copy-pasting, a clear signal of [code-smell](https://en.wikipedia.org/wiki/Code_smell).

These programs are _monolythic_, in the sense that they are not made by composing large enough modules. In this case, the common logic of the program and the values it operates on are intermingled in the same code. 

How can we abstract away the differences and package the common logic in a single module? With functions:

In [12]:

def getValue(key: String, 
             default: String,
             config: Map[String, String]): String = 
    config.get(key) match {
      case Some(p) => p
      case None => default
    }


defined function getValue

This is an abstract module which we can combine with other modules to get back the very same functionality:

In [14]:
getValue

cmd14.sc:1: missing argument list for method getValue in class Helper
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `getValue _` or `getValue(_,_,_)` instead of `getValue`.
val res14 = getValue
            ^Compilation Failed

: 

In [14]:
// Program 1
val port: String = getValue("DBNAME","test",config)

port: String = "test"

In this case, we combine the module `getKeyFrom` with the modules (data values and variables, in particular) `config`, `"URL"` and `"default.url"`. The composition method is just simple function application.

Which are the advantages of using functions? As in the general case, having a more modular solution enables _reuse_, particularly of those modules which are abstract or parameterised. For instance, we can benefit from this level of reuse by re-implementing the `url` program in the following way:

In [3]:
// Program 2
// val port: String = ???

## Functions as methods

In an object-oriented language, functions are implemented through _methods_, i.e. using the `def` keyword. Note that these methods are invariably part of an `object`, `class` or `trait` declaration. Typically, pure functions are declared as part of objects. For instance, we may declare a set of arithmetic functions as follows: 

In [19]:
import scala.math.{pow, Pi}

object Areas{
    
    def circle(radius: Double): Double = 
        Pi * pow(radius, 2)
    
    def rectangle(width: Double, height: Double): Double = 
        width * height
}

import scala.math.{pow, Pi}


defined object Areas

In [20]:
Areas.circle(1)

res19: Double = 3.141592653589793

In notebooks and the Scala REPL, `def` declarations appear to be independent from any object or class, but they are not:

In [5]:
def foo(i: Int): Int = i+1
// show errors: "missing argument list for method foo in class Helper"


defined function foo

When we study higher-order functions, we will see that functions in Scala can also be represented as _objects_, i.e. not only as methods. However, that representation also builds essentially upon methods.

## Functions as values

Functions can also be represented as _values_, i.e. as objects. This allows us to implement functions that receive other functions as arguments, or return functions as results. This special functions are called _higher-order functions_ (HOF), and they feature as a great modularity device. We will mainly discuss this feature of HOFs in PF-3.

In order to represent functions as values, we need first to make extremely clear the difference between variables, values and types. For instance:


In [31]:
// three variables
val i: Int = 1
val s: String = ""
val b: Boolean = true
val j = 2

def fooM(i: Int): Int = 
    i+1

val fooV: Int => Int = 
    (i: Int) => i+1 : Int

i: Int = 1
s: String = ""
b: Boolean = true
j: Int = 2
defined function fooM
fooV: Int => Int = ammonite.$sess.cmd30$Helper$$Lambda$2215/0x00000008015dc840@59aa2dad

In these definitions, we found three variables: `i`, `s` and `b`. These variables are assigned three __values__: `3`, `"hi"` and `true`. The __types__ of these values are, respectively: `Int`, `String` and `Boolean`. Now, let's consider these other variables that we intend to represent values equivalent to the following function-methods:

In [22]:
// Function-methods

def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

// Function-values


val addOne: Int => Int = 
    (number: Int) => number + 1

val substractOne: Int => Int = 
    (number: Int) => number - 1 
    


defined function addOneM
defined function substractOneM
addOne: Int => Int = ammonite.$sess.cmd21$Helper$$Lambda$2171/0x00000008015bf040@3a41fb20
substractOne: Int => Int = ammonite.$sess.cmd21$Helper$$Lambda$2172/0x00000008015be840@156b2c19

In [27]:
val substractOne: Int => Int = 
    (number: Int) => number - 1 : Int

substractOne: Int => Int = ammonite.$sess.cmd26$Helper$$Lambda$2199/0x00000008015d2840@72095403

In [28]:
val substractOne: Int => Int = 
    (number: Int) => number - 1

substractOne: Int => Int = ammonite.$sess.cmd27$Helper$$Lambda$2203/0x00000008015d5040@1a660924

In [29]:
val substractOne: Int => Int = 
    number => number - 1

substractOne: Int => Int = ammonite.$sess.cmd28$Helper$$Lambda$2207/0x00000008015d7840@6b6c9e76

In [32]:
val substractOne = 
    (number: Int) => number - 1

substractOne: Int => Int = ammonite.$sess.cmd31$Helper$$Lambda$2227/0x00000008015e3040@55cdba8a

In [34]:
val substractOne = 
    number => number - 1

substractOne: Int => Int = ammonite.$sess.cmd33$Helper$$Lambda$2238/0x00000008015e9840@59b7bc0e

In [30]:
val substractOne: Int => Int = 
    _ - 1

substractOne: Int => Int = ammonite.$sess.cmd29$Helper$$Lambda$2211/0x00000008015da040@548a7719

In [25]:
addOneM(1)
addOneM(2)
addOne(1)
addOne(2)

res24_0: Int = 2
res24_1: Int = 3
res24_2: Int = 2
res24_3: Int = 3

Here, we also have variables, values and types. The two variables are named `addOneV` and `substractOneV`. They are assigned the _function values_ `(a: Int) => a+1` and `(a: Int) => a - 1`. The type of these values is the same _function type_ `Int => Int`. A function-value is also known as a _lambda expression_. Note that a function value is made of two parts: the input arguments and the function body: `(...input...) => body`. The input arguments declare new variables, each of them of a particular type, that will be assigned to certain values when they are passed to the function (upon invocation). The function body has to be an expression of the type specified as output by the function type.

Function-values are equivalent to function-methods in the sense that they behave exactly in the same way, i.e. they allow us to compute values from other values that we pass as input:

In [6]:
// function-method invocations
// addOneV(5)
// addOneM(5)
// equivalent invocations with function-values


But then, which are the advantages of function values? Basically, they allow us to implement HOFs. For instance, let's say that we want to implement a HOF that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and calls this function over a given number. We may want to write something like this:

In [6]:
def call(def int2int(n: Int): Int, number: Int): Int =
    int2int(number)

where the first argument `int2int` attempts to represent any function that receives an integer and returns another integer. 

But this code is not legal in Scala, because arguments to functions need to be values, not methods. That's why we need function-values!

In [36]:
def call(int2int: Int => Int, number: Int): Int =
    int2int(number)

defined function call

which we can use as follows:

In [6]:
// invoke call
call(??? : Int => Int, ??? : Int)

In [38]:
// invoke call
call(addOne : Int => Int, 4 : Int)

res37: Int = 5

In [39]:
// invoke call
call(((x: Int) => 2*x) : (Int => Int), 5 : Int)

res38: Int = 10

In [39]:
// invoke call
call((x: Int) => 2*x, 5)

res38: Int = 10

In [40]:
// invoke call
call(x => 2*x, 5)

res39: Int = 10

In [41]:
// invoke call
call(2*_, 5)

res40: Int = 10

And we can even pass function-methods that are converted on the fly to function-values!

And we can even pass function-methods that are converted on the fly to function-values!

In [45]:
// invoke call with function-methods
call((x: Int) => x-1, 6)
call(x => addOneM(x), 6)
call(addOneM, 6)

res44_0: Int = 5
res44_1: Int = 7
res44_2: Int = 7

This conversion is the so-called _eta-expansion_.

## Syntactic sugar for function-values

We discuss now some syntactic facilities offered by Scala when writing lambda expressions. 

First, we can omit the types of input arguments and let Scala figure out them:

In [7]:
val addOneV: Int => Int = 
    (a: Int) => a + 1

val substractOneV: Int => Int = 
    (a: Int) => a - 1

addOneV: Int => Int = ammonite.$sess.cmd6$Helper$$Lambda$2025/1323160647@5d49d295
substractOneV: Int => Int = ammonite.$sess.cmd6$Helper$$Lambda$2026/1092195202@6d435536

Second, we can get extra level of conciseness using so-called _underscore_ syntax:

In [8]:
val addOne: Int => Int = 
    (a: Int) => a + 1

addOne: Int => Int = ammonite.$sess.cmd7$Helper$$Lambda$2038/1633876334@1aa2c818

In [8]:
// call((a: Int) => a + 1, 5)
// call((a: Int) => a - 1, 3)

## Currying

What about functions that receive more than one argument? We would like to implement the function-value equivalent of this function-method: 

In [8]:
// function-method sum
def sumM(x: Int, y: Int): Int = 
    x+y

def sum3(x: Int, y: Int, z: Int) : Int = 
    x+y+z


We do that as follows:

In [48]:
// function-value sum
val sum: (Int, Int) => Int = 
    (x: Int, y: Int) => x+y

val sum3V: (Int, Int, Int) => Int = 
    (x,y,z) => x+y+z

val sum3V_ : (Int, Int, Int) => Int = 
    _+_+_

sum: (Int, Int) => Int = ammonite.$sess.cmd47$Helper$$Lambda$2315/0x000000080161f040@a1efd5e
sum3V: (Int, Int, Int) => Int = ammonite.$sess.cmd47$Helper$$Lambda$2316/0x000000080161ec40@62855ca3
sum3V_: (Int, Int, Int) => Int = ammonite.$sess.cmd47$Helper$$Lambda$2317/0x000000080161f840@c535812

In [52]:

def callM(int2int: Int => Int, i: Int): Int = 
    int2int(i)

val call: (Int => Int, Int) => Int = 
    (int2int: Int => Int, i: Int) => int2int(i)

val call2: (Int => Int, Int) => Int = 
    (int2int, i) => int2int(i)

val call3: (Int => Int, Int) => Int = 
    _(_)

defined function callM
call: (Int => Int, Int) => Int = ammonite.$sess.cmd51$Helper$$Lambda$2344/0x0000000801631040@5be8c5ca
call2: (Int => Int, Int) => Int = ammonite.$sess.cmd51$Helper$$Lambda$2345/0x0000000801632040@29b60806
call3: (Int => Int, Int) => Int = ammonite.$sess.cmd51$Helper$$Lambda$2346/0x0000000801632840@7c86335

or, exploiting type inference:

In [8]:
// function-value sum, with type-inference & underscore syntax

However, function types of two, three, ... arguments are not extrictly necessary, and sometimes we can get along with functions of one argument. But, how can we create a function of two arguments with functions of one argument alone? The trick is the following:

In [ ]:
val f: X => Y = 
    (x: X) => ??? : Y

In [61]:
// function-value sum, currified

val sum: (Int, Int) => Int = 
    (x,y) => x+y

val sumC: Int => (Int => Int) = 
    (x: Int) => 
        ((y: Int) => 
             x+y : Int) : (Int => Int)

sum: (Int, Int) => Int = ammonite.$sess.cmd60$Helper$$Lambda$2419/0x0000000801661840@1ffe035a
sumC: Int => Int => Int = ammonite.$sess.cmd60$Helper$$Lambda$2420/0x0000000801662040@416e01dd

In [61]:
sum(2)

cmd61.sc:1: not enough arguments for method apply: (v1: Int, v2: Int)Int in trait Function2.
Unspecified value parameter v2.
val res61 = sum(2)
               ^Compilation Failed

: 

Note that brackets in `Int => (Int => Int)` are used for clarity, but are not needed. Basically, we created a function of one argument that returns another function of one argument. So, the expression: 

In [64]:
// partial application
(y: Int) => sum(1, y)
val f: Int => Int = sumC(1)

// total application
sum(1,2)
sumC(1)(2)

res63_0: Int => Int = ammonite.$sess.cmd63$Helper$$Lambda$2447/0x000000080167a840@55f066a6
f: Int => Int = ammonite.$sess.cmd60$Helper$$Lambda$2430/0x0000000801670840@6e0639a7
res63_2: Int = 3
res63_3: Int = 3

returns a function that can be applied again:

In [67]:
// total application
call(sumC(1), 6)
call(y => sum(1, y), 6)
call(sum(1,_), 6)

res66_0: Int = 7
res66_1: Int = 7
res66_2: Int = 7

We can apply this strategy to functions of any number of arguments. This is called _currying_ and _currified functions_. The analog in function-methods is [multiple-parameter lists](https://docs.scala-lang.org/tour/multiple-parameter-lists.html):

In [2]:
// function-method, with multi-parameter list
def sumM(x: Int, y: Int): Int = 
    x+y

val sum: (Int, Int) => Int = 
    (x, y) => x+y

def sumMC(x: Int)(y: Int): Int = 
    x+y


val sumC: Int => (Int => Int) = 
    x => y => x+y

defined function sumM
sum: (Int, Int) => Int = ammonite.$sess.cmd1$Helper$$Lambda$1905/0x00000008014ef040@1a02485
defined function sumMC
sumC: Int => Int => Int = ammonite.$sess.cmd1$Helper$$Lambda$1906/0x00000008014ee040@2541c38c

In [5]:
val f: Int => Int = sumMC(5)
val g: Int => (Int => Int) = sumMC

f: Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$1969/0x0000000801522040@5d051526
g: Int => Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$1970/0x0000000801523040@280e39a2

## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [6]:
// isEvenLength
def isEvenLength(x: String): Boolean = 
    x.length % 2 == 0

defined function isEvenLength

In [8]:
isEvenLength("abc")

res7: Boolean = false

This function is somehow the combination of two more basic functions `length` and `isEven`:

In [9]:
// length 
(x: String) => x.length

res8: String => Int = ammonite.$sess.cmd8$Helper$$Lambda$2000/0x000000080153b040@1481cc72

In [12]:
// isEven
def isEven(x: Int): Boolean =
    x % 2 == 0

def isOdd(x: Int): Boolean = 
    ! isEven(x)

defined function isEven
defined function isOdd

In [11]:
// isEvenLength
def isEvenLength(x: String): Boolean = 
    isEven(x.length)

defined function isEvenLength

In [11]:
// isOddLength
def isOddLength(x: String): Boolean = 
    isOdd(x.length)

defined function isEvenLength

but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` using the functions `length` and `isEven`? We can use a HOF which helps us to compose functions:

In [17]:
// compose HOF
def compose(g: Int => Boolean, f: String => Int)(x: String): Boolean = 
    g(f(x))

defined function compose

In [14]:
// compose HOF
def compose(g: Int => Boolean, f: String => Int): String => Boolean = 
    x => g(f(x))

defined function compose

In [15]:
// compose HOF
val compose: (Int => Boolean, String => Int) => (String => Boolean) = 
    (g, f) => x => g(f(x))

compose: (Int => Boolean, String => Int) => String => Boolean = ammonite.$sess.cmd14$Helper$$Lambda$2008/0x000000080153f840@2f079954

In [16]:
// compose HOF
val compose: (Int => Boolean) => (String => Int) => (String => Boolean) = 
    g => f => x => g(f(x))

compose: Int => Boolean => String => Int => String => Boolean = ammonite.$sess.cmd15$Helper$$Lambda$2016/0x0000000801545040@29105b85

Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [19]:
def isOddLength(x: String): Boolean = 
    isOdd(x.length)

val isOddLength: String => Boolean = 
    compose(isOdd, _.length)

defined function isOddLength
isOddLength: String => Boolean = ammonite.$sess.cmd18$Helper$$Lambda$2036/0x0000000801552040@44e5cccf

The HOF `compose` is actually defined for function types of one argument: 

In [22]:
val isEvenLength: String => Boolean = 
    (isEven _) compose ((x: String) => x.length)

isEvenLength: String => Boolean = scala.Function1$$Lambda$2043/0x0000000801556840@7c51de5c

or using infix notation:

In [8]:
// val isEvenLength: String => Boolean = ???

Note that a similar function to `compose`, called `andThen`, is also available in the standard library: 

In [8]:
// val isEvenLength: String => Boolean = ???

The last implementation of the `compose` HOF was _monomorphic_, in the sense that it only works with specific types. We can obtain a more flexible implementation using generics, also known as _parametric polymorphism_:

In [23]:
def compose(f2: Int => Boolean, f1: String => Int): String => Boolean = 
    a => f2(f1(a))

defined function compose

In [24]:
def compose(f2: Int => String, f1: Char => Int): Char => String = 
    a => f2(f1(a))

defined function compose

In [26]:
def compose[A, B, C](f2: B => C, f1: A => B): A => C = 
    a => f2(f1(a)): C

defined function compose

In [29]:
compose[String, Int, Boolean](isEven, _.length)
compose(isEven, (x: String) => x.length)
//compose(isEven, x => x.length)

res28_0: String => Boolean = ammonite.$sess.cmd25$Helper$$Lambda$2071/0x000000080156e040@15fb916b
res28_1: String => Boolean = ammonite.$sess.cmd25$Helper$$Lambda$2071/0x000000080156e040@57ddae6b
res28_2: String => Boolean = ammonite.$sess.cmd25$Helper$$Lambda$2071/0x000000080156e040@21d5644b

We can also give a currified version of this function as follows:

In [30]:
// compose currified
def compose[A, B, C]: (B => C, A => B) => A => C = 
    (f2, f1) => a => f2(f1(a)): C

defined function compose

In [31]:
// compose currified
def compose[A, B, C]: (B => C) => (A => B) => A => C = 
    f2 => f1 => a => f2(f1(a)): C

defined function compose

In [31]:
// compose currified
//val compose: [A, B, C] =>> (B => C) => (A => B) => A => C = 
  //  f2 => f1 => a => f2(f1(a)): C

defined function compose

Last, there is a function which behaves as the identity element with respect to the operation `compose`, i.e. no matter which other function we choose to compose with the [`identity`](https://www.scala-lang.org/api/current/scala/Predef$.html) function, the result will be that function:
1. `identity[B] compose f == f` for all `f: A => B`
2. `f compose identity[A] == f` for all `f: A => B`

1 * a == a for all a: Int

a * 1 == a for all a: Int

In [9]:
// function-method identity
identity[Boolean] compose isEven == isEven
isEven compose identity[Int] == isEven

or using lambda expressions:

In [32]:
// quasi function-value identity
def identity[A](a: A): A = 
    a : A

defined function identity

In [33]:
// quasi function-value identity
def identity[A](a: A): A = 
    1.asInstanceOf[A]

defined function identity

In [38]:
// quasi function-value identity
def identity[A](a: A): A = 
    if (a.isInstanceOf[String]) "".asInstanceOf[A]
    else a

defined function identity

In [39]:
identity[String]("adasf")

res38: String = ""

In [36]:
// quasi function-value identity
def identity[A](a: A): A = 
    throw new Exception("")

defined function identity

In [37]:
identity[String]("")

: 

In [ ]:
def f(b: Boolean): Boolean = 
    !b

In [ ]:
val f: String => Boolean = 
    (x: String) => ??? : Boolean

In [41]:
def foo[A, B, C](g: B => C, f: A => B): A => C = 
    ??? : (A => C)

defined function compose

In [42]:
def foo[A, B, C](g: B => C, f: A => B): A => C = 
    ((a: A) => ??? : C): (A => C)

defined function foo

In [43]:
def foo[A, B, C](g: B => C, f: A => B): A => C = 
    ((a: A) => g(f(??? : A) : B) : C): (A => C)

defined function foo

In [44]:
def foo[A, B, C](g: B => C, f: A => B): A => C = 
    ((a: A) => g(f(a : A) : B) : C): (A => C)

defined function foo

In [45]:
def foo[A, B, C](g: B => C, f: A => B): A => C = 
    a => g(f(a))

defined function foo

## How are functions represented as values

Now, we just want to focus on how are functions actually represented as values in a OO language like Scala. This representation builds essentially upon methods, in particular, _reified_ methods. For instance, let's consider the following functions:

In [10]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

defined function addOneM
defined function substractOneM

In order to create a type of functions that receive an integer and return another one, we can create a new class whose only method is the function that we want to actually implement:

In [53]:
// FunctionInt2Int class 
val addOneV: Int => Int = 
    (x: Int) => x+1

addOneV: Int => Int = ammonite.$sess.cmd52$Helper$$Lambda$2172/0x00000008015b3840@2a925ced

In [49]:
// FunctionInt2Int class 
val addOne: Function1[Int, Int] = 
    (x: Int) => x+1

addOne: Int => Int = ammonite.$sess.cmd48$Helper$$Lambda$2137/0x000000080159e040@4f21ba60

In [50]:
// FunctionInt2Int class 
val addOne: Function1[Int, Int] = 
    new Function1[Int, Int]{
        def apply(x: Int): Int =
            x+1
    }

addOne: Int => Int = <function1>

In [ ]:
object std{
    trait Function1[-A, +B]{
        def apply(x: A): B
    }
}

In [54]:
// FunctionInt2Int class 
object addOne extends Function1[Int, Int]{
    def apply(x: Int): Int =
        x+1
}


defined object addOne

In [58]:
addOne(5)

res57: Int = 6

In [56]:
addOneV(5)

res55: Int = 6

In [51]:
addOne.apply(5)

res50: Int = 6

Now, we can implement the `call` HOF as follows: 

In [10]:
// call HOF


In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [10]:
// addOneV and SubstractV function-values


We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

In [10]:
// invoke call HOF

Actually, function types such as `Int => Int` and `Boolean => String` are syntactic sugar for the types `Function1[Int, Int]` and `Function1[Boolean, String]`, where [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html) is a generalization of the type `FunctionInt2Int` that we wrote above. We have also [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), [`Function3`](https://www.scala-lang.org/api/current/scala/Function3.html), etc., that are roughly implemented as follows:

In [11]:
object Std{
    trait Function1[A, B]{
        def apply(a: A): B
    }

    trait Function2[A, B, C]{
        def apply(a: A, b: B): C
    }

    // up to Function22
}


defined object Std

Using these standard classes, we can create the `addOneV` function-value in a similar way than before: 

In [11]:
// addOneV as a Function1 instance


and invoke functions as follows:


In [11]:
// invoke addOneV


However, as we saw throughout this notebook, we can also invoke the function without explicitly naming the `apply` method, i.e. 

In [11]:
// omit apply!


This is just another syntactic nicety of Scala. In sum, in an object-oriented language like Scala, function-values are ultimately methods in disguise.